In [7]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [8]:
ds = pd.read_csv('churn.csv', index_col=0)
ds.replace('?', np.nan, inplace=True)


churn = pd.get_dummies(ds['Churn?'], prefix='churn', drop_first=True)
state = pd.get_dummies(ds['State'], prefix='state', drop_first=True)
intplan = pd.get_dummies(ds["Int'l Plan"], prefix='int_plan', dummy_na=True)
vmail = pd.get_dummies(ds['VMail Plan'], prefix='vmail_plan', dummy_na=True)
ds['know_areaCode'] = pd.notnull(ds['Area Code'])

ds.drop(['Churn?', 'State', "Int'l Plan", 'VMail Plan', 'Area Code','Phone'], axis=1, inplace=True)
df = pd.concat([ds, churn, state, intplan, vmail], axis=1)

In [9]:
y = df['churn_True.']
X = df.drop('churn_True.', axis=1)
imp = Imputer(missing_values='NaN', strategy='median')
X= imp.fit_transform(X)

In [10]:
testModel = XGBClassifier()
testModel = testModel.fit(X, y)
print testModel.score(X,y)
#the overall fit seems to do pretty well, now implement 10-kfolds and test AUC

0.962196219622


In [11]:
kf = KFold(n_splits=10, shuffle=True)
testPreds = np.zeros_like(y, dtype=float)
for train_set, test_set in kf.split(X):
    model = XGBClassifier()
    model = model.fit(X[train_set], y[train_set])
    testPreds[test_set] = model.predict_proba(X[test_set])[:,1]   
print roc_auc_score(y, testPreds)

0.908448657877
